In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/RecSys/data.csv')

### Анализ

Анализ будет по большей части повторять то, что было сделано в первом ДЗ. 

Удалим ненужные колонки и сравним banner_id и banner_id0, как от нас просят в задании.

In [ ]:
to_drop = ["campaign_clicks", "oaid_hash", "rate0", "rate1"]
data = data.drop(to_drop, axis=1)
data = data[data['banner_id0'] == data['banner_id']]
data.head()

,date_time,zone_id,banner_id,os_id,country_id,banner_id0,g0,coeff_sum0,banner_id1,g1,coeff_sum1,impressions,clicks
1,2021-09-26 22:54:49.000000,1,1,0,1,1,0.054298,-2.657477,269,0.031942,-4.449220,1,1
2,2021-09-26 23:57:20.000000,2,2,0,0,2,0.014096,-3.824875,21,0.014906,-3.939309,1,1
3,2021-09-27 00:04:30.000000,3,3,1,1,3,0.015232,-3.461357,99,0.050671,-3.418403,1,1
4,2021-09-27 00:06:21.000000,4,4,1,0,4,0.051265,-4.009026,11464230,0.032005,-2.828797,1,1
5,2021-09-27 00:06:50.000000,5,5,2,2,5,0.337634,-3.222757,37,0.338195,-3.221755,1,1


Посмотрим на столбец impressions, пока кажется, что он не несет информации.

In [ ]:
data.groupby("impressions").count()

,date_time,zone_id,banner_id,os_id,country_id,banner_id0,g0,coeff_sum0,banner_id1,g1,coeff_sum1,clicks
impressions,,,,,,,,,,,,
1,13947160,13947160,13947160,13947160,13947160,13947160,13947160,13947160,13947160,13927485,13927485,13947160


In [ ]:
data = data.drop("impressions", axis=1)

Проверим, есть ли None:

In [ ]:
data.isnull().value_counts()

date_time  zone_id  banner_id  os_id  country_id  banner_id0  g0     coeff_sum0  banner_id1  g1     coeff_sum1  clicks
False      False    False      False  False       False       False  False       False       False  False       False     13927485
                                                                                             True   True        False        19675
dtype: int64

Как видим, в новых колонках есть None, удалим их.

In [ ]:
data = data.dropna()
data.isnull().value_counts()

date_time  zone_id  banner_id  os_id  country_id  banner_id0  g0     coeff_sum0  banner_id1  g1     coeff_sum1  clicks
False      False    False      False  False       False       False  False       False       False  False       False     13927485
dtype: int64

Посмотрим, соответствует ли g1 и g0 свойствам дисперсии, удалим ненужное.

In [ ]:
print(f"g0 < 0: {len(data[data['g0'] < 0])} items, g1 < 0: {len(data[data['g1'] < 0])} items")
data = data[(data['g1'] > 0) & (data['g0'] > 0)]

g0 < 0: 5 items, g1 < 0: 5 items


In [ ]:
data["date_time"] = pd.to_datetime(data["date_time"])
data['date'] = data.date_time.dt.date
data.date.value_counts()

2021-09-26    2638356
2021-09-29    2103643
2021-09-27    2019851
2021-09-28    1947157
2021-10-02    1853252
2021-09-30    1594244
2021-10-01    1413503
2021-09-01          1
Name: date, dtype: int64

По одному из дней только одна запись, выбросим его.

In [ ]:
import datetime
data = data.drop(data[data["date"] == datetime.date(2021, 9, 1)].index.asi8)
data = data.sort_values(by='date')
data.date.value_counts()

<ipython-input-11-17fc0e0f6986>:2: FutureWarning: Index.asi8 is deprecated and will be removed in a future version
  data = data.drop(data[data["date"] == datetime.date(2021, 9, 1)].index.asi8)


2021-09-26    2638356
2021-09-29    2103643
2021-09-27    2019851
2021-09-28    1947157
2021-10-02    1853252
2021-09-30    1594244
2021-10-01    1413503
Name: date, dtype: int64

In [ ]:
data.columns

Index(['date_time', 'zone_id', 'banner_id', 'os_id', 'country_id',
       'banner_id0', 'g0', 'coeff_sum0', 'banner_id1', 'g1', 'coeff_sum1',
       'clicks', 'date'],
      dtype='object')

### Feature engineering

Применим One hot encoding:

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack

def feature_engineering(data: pd.DataFrame):

    data = data.drop(['banner_id0'], axis=1)

    #делим на train/test для логистической регрессии
    train = data[data["date"] < datetime.date(2021, 10, 2)]
    test = data[data["date"] == datetime.date(2021, 10, 2)]

    x_train = train.drop(["clicks", "date", "date_time", "banner_id1", "g0", "g1", "coeff_sum0",  "coeff_sum1"], axis=1)
    y_train = np.array(train["clicks"])
    x_test = test.drop(["clicks", "date", "date_time", "banner_id1", "g0", "g1", "coeff_sum0",  "coeff_sum1"], axis=1)
    y_test = np.array(test["clicks"])

    #преобразуем данные
    #используем разреженные матрицы, поскольку OHE преимущественно нули
    ohe = OneHotEncoder(handle_unknown='ignore', sparse=True)
    print("X test:", x_test.columns)
    x_train = ohe.fit_transform(x_train)
    print("OHE train transformed")
    x_test = ohe.transform(x_test)

    #для дальнейших вычислений найдем характеристики распределений за последний день
    distr_char = test[["g0", "g1", "coeff_sum0", "coeff_sum1"]]

    #сформируем тестовые выборки для обучения 
    test = test.drop(["clicks", "date", "date_time", "g0", "g1", "coeff_sum0",  "coeff_sum1"], axis=1)
    x_test_pi0 = test.drop(["banner_id1"], axis=1)
    test["banner_id"] = test["banner_id1"]
    x_test_pi1 = test.drop(["banner_id1"], axis=1)
    print("X test pi0:", x_test_pi0.columns)
    print("X test pi1:", x_test_pi1.columns)
    x_test_pi0 = ohe.transform(x_test_pi0)
    x_test_pi1 = ohe.transform(x_test_pi1) 
  
    return x_train, x_test, y_train, y_test, x_test_pi0, x_test_pi1, distr_char

In [ ]:
x_train, x_test, y_train, y_test, x_test_pi0, x_test_pi1, distr_char = feature_engineering(data)

X test: Index(['zone_id', 'banner_id', 'os_id', 'country_id'], dtype='object')
OHE train transformed
X test pi0: Index(['zone_id', 'banner_id', 'os_id', 'country_id'], dtype='object')
X test pi1: Index(['zone_id', 'banner_id', 'os_id', 'country_id'], dtype='object')


In [ ]:
from sklearn.linear_model import LogisticRegression

### Модель

Будем использовать линейную регрессию из первого ДЗ.

In [ ]:
def create_model(penalty='l1', C=1.0):
    model = LogisticRegression(penalty=penalty, C=C, solver='liblinear')
    return model

In [ ]:
model = create_model(C=0.01, penalty='l2')
model.fit(x_train, y_train)

LogisticRegression(C=0.01, solver='liblinear')

In [ ]:
from sklearn.metrics import log_loss, roc_auc_score
y_model = model.predict_proba(x_test)[:, 1]
y_base = np.mean(y_test)*np.ones(y_test.shape)
print(f"My model: NNL = {log_loss(y_test, y_model)}, ROC AUC = {roc_auc_score(y_test, y_model)}")
print(f"Baseline: NNL = {log_loss(y_test, y_base)}, ROC AUC = {roc_auc_score(y_test, y_base)}")

My model: NNL = 0.13428184350706243, ROC AUC = 0.7911949175263264
Baseline: NNL = 0.15574130602526282, ROC AUC = 0.5


Посчитаем суммы коэффициентов coeff_sum0_new и coeff_sum1_new из предиктов нашей модели.

In [ ]:
def logit(x):
    return np.log(x) - np.log(1 - x)

coeff_sum0_new = logit(model.predict_proba(x_test_pi0)[:, 1])
coeff_sum1_new = logit(model.predict_proba(x_test_pi1)[:, 1])


Как найти вероятность того, что одна случайная величина больше другой? 

Пусть $x \sim N(a_1, σ_1)$, $y \sim N(a_2, σ_2)$. Тогда
$$P(x < y) = P(x -y < 0) = F_\xi(0), \ \xi = x -y $$
$$ \xi \sim N(a_2-a_1, \sqrt{σ_1^2 + σ_2^2})$$

Используем функцию нормального распределениия:

In [ ]:
import scipy.stats as st
pi_0 = st.norm.cdf((distr_char['coeff_sum1'] - distr_char['coeff_sum0']) / np.sqrt(distr_char['g0'] ** 2 + distr_char['g1'] ** 2 +1e-7))
pi_1 = st.norm.cdf((coeff_sum1_new - coeff_sum0_new) / np.sqrt(distr_char['g0'] ** 2 + distr_char['g1'] ** 2 + 1e-7))
print(pi_0, pi_1)

[5.50491921e-01 1.02562859e-01 5.00753936e-01 ... 1.90195822e-09
 3.19659705e-01 3.26668334e-01] [4.71296456e-01 3.18811936e-03 5.29390135e-01 ... 2.41008416e-06
 3.20334741e-01 6.69870412e-05]


Посчитаем CIPS:

In [ ]:
cips = np.sum(y_test * np.clip(pi_1/(pi_0+1e-8), 0, 10)) / len(y_test)

In [ ]:
cips

0.09713394822680353